In [1]:
%pip install crewai langchain_openai langchain_community python-dotenv


  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:

from bs4 import BeautifulSoup


In [2]:

from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain.tools import tool
import re
from langchain_community.document_loaders import PyMuPDFLoader
import requests


In [8]:
# Load your OPENAI_API_KEY from your .env file
load_dotenv()
# The model for the agents
model = ChatOpenAI(model_name="gpt-4o", temperature=0.8)
original_url = "http://aiyaayia.com"
@tool
def get_category_urls() -> str:
    """
    Reads the webpage with the original URL and returns the category page URLs.
    """
    try:
        response = requests.get(original_url)
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Assuming category links are within <a> tags with a specific class or attribute
        category_links = []
        for link in soup.find_all('a', class_='category-link'):  # Update the class name accordingly
            category_url = link.get('href')
            if category_url:
                category_links.append(category_url)
        
        return '\n'.join(category_links)
    except requests.exceptions.RequestException as e:
        return str(e)
# Agents

agent_semantic_category_analyzer = Agent(
    role='Semantic Category Analyzer',
    goal='Identify semantically relevant categories',
    backstory="""You are responsible for analyzing all categories on the live website 
    and finding semantically relevant categories to the one we are writing about. 
    You need to return four categories that are semantically related to the category 
    as well as the links for each category.""",
    tools=[get_category_urls],  # Add the updated custom scraping tool
    llm=model,
    max_iter=15,
    verbose=True,
    allow_delegation=True,
    cache=True
)

agent_ecommerce_content_creator = Agent(
    role='E-commerce Content Creator',
    goal='Write comprehensive text for the category',
    backstory="""You are responsible for receiving information about the website 
    and the category we are writing the description for. You then write a comprehensive 
    and engaging text for the category.""",
    llm=model,
    max_iter=15,
    verbose=True,
    allow_delegation=True,
    cache=True
)

agent_link_integration_specialist = Agent(
    role='Link Integration Specialist',
    goal='Create unique interlinking sentences',
    backstory="""You are responsible for receiving the category links and text, 
    creating four unique interlinking sentences. Each sentence must have one 
    semantically related category name linked to its respective URL provided 
    by the Semantic Category Analyzer. You must avoid redundancy and overlap.""",
    tools=[get_category_urls],  # Add the updated custom scraping tool
    llm=model,
    max_iter=15,
    verbose=True,
    allow_delegation=True,
    cache=True
)

agent_content_integration_manager = Agent(
    role='Content Integration Manager',
    goal='Combine text and interlinking sentences into a cohesive description',
    backstory="""You are responsible for receiving the category text from 
    the E-commerce Content Creator and the interlinking sentences from the 
    Link Integration Specialist. You need to combine them into a cohesive 
    and engaging category description.""",
    tools=[get_category_urls],  # Add the updated custom scraping tool
    llm=model,
    max_iter=15,
    verbose=True,
    allow_delegation=True,
    cache=True
)


# Tasks


In [10]:


# Task for analyzing categories
task_analyze_categories = Task(
    description='Analyze categories on the live website and identify semantically relevant ones',
    expected_output='Four categories that are semantically related along with their links',
    agent=agent_semantic_category_analyzer,
    tools=[get_category_urls]  # Specify the custom scraping tool
)

# Task for writing category text
task_write_category_text = Task(
    description='Write a comprehensive text for the category',
    expected_output='A well-written category text',
    agent=agent_ecommerce_content_creator
    # No tools specified, assuming text generation is done by LLM
)

# Task for creating interlinking sentences
task_create_interlinking_sentences = Task(
    description='Create four unique interlinking sentences for the category text',
    expected_output='Four interlinking sentences that fit the category text',
    agent=agent_link_integration_specialist,
)

# Task for integrating content
task_integrate_content = Task(
    description='Combine the category text and interlinking sentences into a cohesive description',
    expected_output='A perfect category description',
    agent=agent_content_integration_manager
    # No tools specified, assuming integration is done by LLM
)


In [17]:
crew = Crew(
    agents=[
        agent_semantic_category_analyzer,
        agent_ecommerce_content_creator,
        agent_link_integration_specialist,
        agent_content_integration_manager
    ],
    tasks=[
        task_analyze_categories,
        task_write_category_text,
        task_create_interlinking_sentences,
        task_integrate_content
    ],
    verbose=True
)

# Start the crew's execution
crew.kickoff(inputs={"topic": "vegan leather", "website": "aiyaayia.com"})

2024-08-06 12:18:13,240 - 8507938048 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-06 12:18:13][DEBUG]: == Working Agent: Semantic Category Analyzer
 [2024-08-06 12:18:13][INFO]: == Starting Task: Analyze categories on the live website and identify semantically relevant ones


> Entering new CrewAgentExecutor chain...
I need to analyze the categories on the live website and identify semantically relevant ones to the current category. To do this, I will first need to read the webpage and extract the category page URLs.

Action: get_category_urls
Action Input: {"url": "https://example.com"} 



Thought: I need to analyze the categories on the live website and identify semantically relevant ones to the current category. To do this, I will first need to read the webpage and extract the category page URLs.

Action: get_category_urls
Action Input: {"url": "https://example.com"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



I realize my mistake in the previous actions. I should have extracted the categor

CrewOutput(raw="---\nWhether you're dreaming of tropical beaches, bustling cities, or serene countryside, our [destinations](https://example.com/destinations) guides will help you choose the perfect spot for your next adventure. From packing lists to safety advice, our [travel tips](https://example.com/travel-tips) section provides practical information to help you prepare for your trip. Dive into our articles on [cultural experiences](https://example.com/cultural-experiences) to learn about local traditions, festivals, cuisine, and more. Additionally, find the best [travel deals](https://example.com/travel-deals) and discounts to make your trip affordable, featuring exclusive offers on flights, hotels, tours, and vacation packages.\n---\n\nThis combined description integrates all the provided interlinking sentences and category text in a cohesive and engaging manner.", pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Analyze categories on the live website and ident